<a href="https://colab.research.google.com/github/yleessam/2501_dl_env_colab/blob/main/10_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
!unzip jena_climate_2009_2016.csv.zip

In [ ]:
import os
fname = os.path.join("jena_climate_2009_2016.csv")

with open(fname) as f:
    data = f.read()

lines = data.split("\n")
header = lines[0].split(",")
lines = lines[1:]
print(header)
print(len(lines))

In [ ]:
import numpy as np
temperature = np.zeros((len(lines),))
raw_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(",")[1:]]
    temperature[i] = values[1]
    raw_data[i, :] = values[:]

In [ ]:
from matplotlib import pyplot as plt
plt.plot(range(len(temperature)), temperature)
plt.show()

각 분할에 사용할 샘플 수 계산하기

데이터 정규화

훈련, 검증, 테스트 데이터셋 만들기

상식적인 기준 모델의 MAE 계산하기

밀집 연결 모델 훈련하고 평가하기

In [ ]:
import matplotlib.pyplot as plt
loss = history.history["mae"]
val_loss = history.history["val_mae"]
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, "bo", label="Training MAE")
plt.plot(epochs, val_loss, "b", label="Validation MAE")
plt.title("Training and validation MAE")
plt.legend()
plt.show()

1D 합성곱 모델

간단한 LSTM 기반 모델

넘파이로 구현한 간단한 RNN

어떤 길이의 시퀀스도 처리할 수 있는 RNN 층

마지막 출력 스텝만 반환하는 RNN 층

전체 출력 시퀀스를 반환하는 RNN 층

스태킹(stacking) RNN 층

과대적합을 감소하기 위해 순환 드롭아웃 사용하기

드롭아웃 규제와 스태킹을 적용한 GRU 모델을 훈련하고 평가하기

양방향 RNN 사용하기